In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import numpy as np

In [ ]:
#Read the data
cars = pd.read_csv("Cars.csv")
cars.head()

In [ ]:
cars.info()

In [ ]:
#check for missing values
cars.isna().sum()

# Correlation Matrix

In [ ]:
sns.heatmap(cars.corr(),annot=True)

# Scatterplot between variables along with histograms

In [ ]:
#Format the plot background and scatter plots for all the variables
sns.set_style(style='darkgrid')
sns.pairplot(cars)

# Preparing a model

In [ ]:
#Build model
import statsmodels.formula.api as smf
model = smf.ols('MPG~WT+VOL+SP+HP',data=cars).fit()

In [ ]:
model.summary()

In [ ]:
#Coefficients
model.params

In [ ]:
#t and p-Values
print(model.tvalues, '\n', model.pvalues)

In [ ]:
#R squared values
(model.rsquared,model.rsquared_adj)

# Simple Linear Regression Models

In [ ]:
ml_v=smf.ols('MPG~VOL',data = cars).fit()
#t and p-Values
print(ml_v.tvalues, '\n', ml_v.pvalues)

In [ ]:
ml_w=smf.ols('MPG~WT',data = cars).fit()
print(ml_w.tvalues, '\n', ml_w.pvalues)

In [ ]:
ml_wv=smf.ols('MPG~WT+VOL',data = cars).fit()
print(ml_wv.tvalues, '\n', ml_wv.pvalues)

# Calculating VIF

In [ ]:
smf.ols("HP~WT+VOL+SP",data=cars).fit().rsquared

In [ ]:
rsq_hp = smf.ols('HP~WT+VOL+SP',data=cars).fit().rsquared
vif_hp = 1/(1-rsq_hp)

rsq_wt = smf.ols('WT~HP+VOL+SP',data=cars).fit().rsquared
vif_wt = 1/(1-rsq_wt)

rsq_vol = smf.ols('VOL~WT+SP+HP',data=cars).fit().rsquared
vif_vol = 1/(1-rsq_vol)

rsq_sp = smf.ols('SP~WT+VOL+HP',data=cars).fit().rsquared
vif_sp = 1/(1-rsq_sp)

# Storing vif values in a data frame
d1 = {'Variables':['Hp','WT','VOL','SP'],'VIF':[vif_hp,vif_wt,vif_vol,vif_sp]}
Vif_frame = pd.DataFrame(d1)
Vif_frame

In [ ]:
model1=smf.ols('MPG~HP+SP+VOL',data=cars).fit()
print(model1.rsquared,model1.rsquared_adj)
model1.summary(())

In [ ]:
model2=smf.ols('MPG~HP+SP+WT',data=cars).fit()
print(model2.rsquared,model2.rsquared_adj)
model2.summary()

In [ ]:
rsq_hp = smf.ols('HP~VOL+SP',data=cars).fit().rsquared
vif_hp = 1/(1-rsq_hp)

rsq_vol = smf.ols('VOL~SP+HP',data=cars).fit().rsquared
vif_vol = 1/(1-rsq_vol)

rsq_sp = smf.ols('SP~VOL+HP',data=cars).fit().rsquared
vif_sp = 1/(1-rsq_sp)

# Storing vif values in a data frame
d1 = {'Variables':['Hp','VOL','SP'],'VIF':[vif_hp,vif_vol,vif_sp]}
Vif_frame = pd.DataFrame(d1)
Vif_frame

# Residual Analysis

## Test for Normality of Residuals (Q-Q Plot)

In [ ]:
model.resid

In [ ]:
import statsmodels.api as sm
qqplot=sm.qqplot(model.resid,line='q') # line = 45 to draw the diagnoal line
plt.title("Normal Q-Q plot of residuals")
plt.show()

In [ ]:
list(np.where(model.resid>10))   #lists out all indices with residual more than 10

## Residual Plot for Homoscedasticity

In [ ]:
def get_standardized_values( vals ):
    return (vals - vals.mean())/vals.std()

In [ ]:
cars['MPG'],model.fittedvalues,model.resid

In [ ]:
plt.scatter(get_standardized_values(model.fittedvalues),
            get_standardized_values(model.resid))

plt.title('Residual Plot')
plt.xlabel('Standardized Fitted values')
plt.ylabel('Standardized residual values')
plt.show()

## Residual Vs Regressors

In [ ]:
fig = plt.figure(figsize=(10,5))            #regression results against one regressor
fig = sm.graphics.plot_regress_exog(model, "VOL", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
fig = sm.graphics.plot_regress_exog(model, "SP", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
fig = sm.graphics.plot_regress_exog(model, "HP", fig=fig)
plt.show()

In [ ]:
fig = plt.figure(figsize=(10,5))
fig = sm.graphics.plot_regress_exog(model, "WT", fig=fig)
plt.show()

# Model Deletion Diagnostics

## Detecting Influencers/Outliers

In [ ]:
model.rsquared

## Cook’s Distance

In [ ]:
model_influence = model.get_influence()
(c, _) = model_influence.cooks_distance

In [ ]:
len(c)

In [ ]:
#Plot the influencers values using stem plot
fig = plt.subplots(figsize=(20, 7))
plt.stem(np.arange(len(cars)), np.round(c, 3))
plt.xlabel('Row index')
plt.ylabel('Cooks Distance')
plt.show()

In [ ]:
#index and value of influencer where c is more than .5
(np.argmax(c),np.max(c))

## High Influence points

In [ ]:
from statsmodels.graphics.regressionplots import influence_plot
influence_plot(model)
plt.show()

In [ ]:
k = cars.shape[1]
n = cars.shape[0]
leverage_cutoff = 3*((k + 1)/n)
print(leverage_cutoff)

#### From the above plot, it is evident that data point 70 and 76 are the influencers

In [ ]:
cars[cars.index.isin([70, 76])]


In [ ]:
#See the differences in HP and other variable values
cars.head()

# Improving the model

In [ ]:
#Load the data
cars_new = pd.read_csv("Cars.csv")

In [ ]:
#Discard the data points which are influencers and reasign the row number (reset_index())
car1=cars_new.drop(cars_new.index[[70,76]],axis=0).reset_index(drop=True)
#car1=cars_new.drop([70,76],axis=0).reset_index()
car1

In [ ]:
#Drop the original index
#car1=car1.drop(['index'],axis=1)

In [ ]:
car1

# Build Model

In [ ]:
#Exclude variable "WT" and generate R-Squared and AIC values
final_ml_V= smf.ols('MPG~VOL+SP+HP',data = car1).fit()

In [ ]:
(final_ml_V.rsquared,final_ml_V.aic)

In [ ]:

#Exclude variable "VOL" and generate R-Squared and AIC values
final_ml_W= smf.ols('MPG~WT+SP+HP',data = car1).fit()

In [ ]:
(final_ml_W.rsquared,final_ml_W.aic)

##### Comparing above R-Square and AIC values, model 'final_ml_V' has high R- square and low AIC value hence include variable 'VOL' so that multi collinearity problem would be resolved.

# Cook’s Distance

In [ ]:
model_influence_V = final_ml_V.get_influence()
(c_V, _) = model_influence_V.cooks_distance

In [ ]:
fig= plt.subplots(figsize=(20,7))
plt.stem(np.arange(len(car1)),np.round(c_V,3));
plt.xlabel('Row index')
plt.ylabel('Cooks Distance');

In [ ]:
#index of the data points where c is more than .5
(np.argmax(c_V),np.max(c_V))

In [ ]:
k = car1.shape[1]-1
n = car1.shape[0]
leverage_cutoff = 3*((k + 1)/n)
print(leverage_cutoff)

In [ ]:
from statsmodels.graphics.regressionplots import influence_plot
influence_plot(final_ml_V)
plt.show()

In [ ]:

#Drop 76 and 77 observations
car2=car1.drop(car1.index[[76,77]],axis=0)

In [ ]:
car2

In [ ]:
#Reset the index and re arrange the row values
car3=car2.reset_index(drop=True)

In [ ]:


#car4=car3.drop(['index'],axis=1)

In [ ]:
car3

In [ ]:
#Build the model on the new data
final_ml_V= smf.ols('MPG~VOL+SP+HP',data = car3).fit()

In [ ]:
final_ml_V.rsquared,final_ml_V.aic

In [ ]:
#Again check for influencers
model_influence_V = final_ml_V.get_influence()
(c_V, _) = model_influence_V.cooks_distance

In [ ]:
fig= plt.subplots(figsize=(20,7))
plt.stem(np.arange(len(car3)),np.round(c_V,3));
plt.xlabel('Row index')
plt.ylabel('Cooks Distance');

In [ ]:
#index of the data points where c is more than .5
(np.argmax(c_V),np.max(c_V))








#### Since the value is <1 , we can stop the diagnostic process and finalize the model

In [ ]:
#Check the accuracy of the mode
final_ml_V= smf.ols('MPG~VOL+SP+HP',data = car3).fit()

In [ ]:
(final_ml_V.rsquared,final_ml_V.aic)

## Predicting for new data

In [ ]:
#New data for prediction
new_data=pd.DataFrame({'HP':40,"VOL":95,"SP":102,"WT":35},index=[1])

In [ ]:
new_data

In [ ]:
final_ml_V.predict(new_data)

In [ ]:
final_ml_V.predict(cars_new.iloc[0:5,])

In [ ]:
cars_new.MPG.head()

In [ ]:
pred_y = final_ml_V.predict(cars_new)

In [ ]:
pred_y

In [ ]:
cars_new.MPG

# **Using Sklearn linear regression**

In [ ]:
import pandas as pd
df=pd.read_csv("Cars.csv")

In [ ]:
df.head()

In [ ]:
X=df.drop(columns=['MPG'])
Y=df.MPG

In [ ]:
from sklearn.linear_model import LinearRegression


In [ ]:
reg=LinearRegression()
reg.fit(X,Y)

In [ ]:
Y_pred=reg.predict(X)

In [ ]:
data=pd.DataFrame({'Actual':Y,
                   'Predicted':Y_pred})
data.head()

In [ ]:
reg.score(X,Y)

In [ ]:
from sklearn.metrics import r2_score
r2_score(Y,Y_pred)

In [ ]:
import sklearn
help(sklearn.metrics)